In [8]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [9]:
from langchain.document_loaders import TextLoader



In [10]:
# loader = TextLoader('docs/FakeWR.txt')
# docs = loader.load()
# len(docs)

#web scrape TTB.gov
from bs4 import BeautifulSoup
import requests

url = 'https://www.ttb.gov/beer/labeling/malt-beverage-mandatory-label-information'
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Successfully fetched the webpage.")
else:
    print("Failed to fetch the webpage.")

soup = BeautifulSoup(response.text, 'html.parser')

print(soup.get_text())

Successfully fetched the webpage.







TTBGov - Malt Beverage Labeling: Mandatory Label Information

































Skip to main content









An official website of the United States government
Here’s how you know


Here’s how you know











                Official websites use .gov


              A .gov website belongs to an official government organization in the United States.

            







                Secure .gov websites use HTTPS


              A lock (
LockA locked padlock
) or https:// means you’ve safely connected to the .gov website. Share sensitive information only on official, secure websites.

            














HomeWho We AreAbout TTBOrganizational ChartTTB OfficesThe TTB StoryOur MissionContact UsTTB OfficesNRC Call CenterALFD Call CenterScientific Services DivisionTrade Investigations DivisionMedia InquiriesCongressional InquiriesSubmit a TipTTB OfficesExecutive StaffOffice of ManagementHeadquarters OperationsField Opera

In [10]:
docs

[Document(page_content="Certainly, tailoring a Workforce Resilience Program specifically for employees of the Office of Information Technology (OIT) at the Centers for Medicare & Medicaid Services (CMS) means integrating elements pertinent to the nature of their work, the demands of the healthcare sector, and the unique challenges faced by IT professionals. Here's how this could be done:\n\n### CMS OIT Workforce Resilience Program\n\n1. **Program Objective**:\n    - Support the continuous operation of CMS digital infrastructure and health data systems.\n    - Align with healthcare regulations and data protection standards.\n    - Foster a resilient IT community capable of responding to healthcare crises.\n\n2. **Risk Assessment**:\n    - Identify IT-specific challenges, such as cybersecurity threats, system outages, and data breaches.\n    - Evaluate potential health sector crises like pandemics, which may strain digital services.\n\n3. **Training & Education**:\n    - Enhance cybersec

In [17]:
# tiktoken_len(docs[0].page_content)
tiktoken_len(soup.get_text())

1983

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,  # number of tokens overlap between chunks
    length_function=tiktoken_len,
    separators=['\n\n', '\n', ' ', '']
)

In [19]:
# chunks = text_splitter.split_text(docs[0].page_content)
chunks = text_splitter.split_text(soup.get_text())

In [20]:
chunks

['TTBGov - Malt Beverage Labeling: Mandatory Label Information\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\n\n\n\n\n\n\n\nAn official website of the United States government\nHere’s how you know\n\n\nHere’s how you know\n\n\n\n\n\n\n\n\n\n\n\n                Official websites use .gov\n\n\n              A .gov website belongs to an official government organization in the United States.\n\n            \n\n\n\n\n\n\n\n                Secure .gov websites use HTTPS\n\n\n              A lock (\nLockA locked padlock\n) or https:// means you’ve safely connected to the .gov website. Share sensitive information only on official, secure websites.',
 'HomeWho We AreAbout TTBOrganizational ChartTTB OfficesThe TTB StoryOur MissionContact UsTTB OfficesNRC Call CenterALFD Call CenterScientific Services DivisionTrade Investigations DivisionMedia InquiriesCongressional InquiriesSubmit a TipTTB OfficesExecutive StaffOffice of ManagementHeadquarters Operat

In [15]:
from tqdm.auto import tqdm
import hashlib

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm.auto import tqdm
import hashlib

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,  # number of tokens overlap between chunks
    length_function=tiktoken_len,
    separators=['\n\n', '\n', ' ', '']
)


m = hashlib.md5()
documents = []

for doc in tqdm(docs):
    source = doc.metadata['source']
    m.update(source.encode('utf-8'))
    uid = m.hexdigest()[:12]
    chunks = text_splitter.split_text(doc.page_content)
    for i, chunk in enumerate(chunks):
        documents.append({
            'id': f'{uid}-{i}',
            'text': chunk,
            'source': source
        })

len(documents)

NameError: name 'docs' is not defined